In [111]:
import random
import re
from urllib.parse import urlparse, parse_qs, parse_qsl, quote

import pandas as pd
import requests

from tqdm import tqdm

from bs4 import BeautifulSoup
import bs4

URL_ROOT = 'https://en.wiktionary.org'

soups = {}
def get_soup(path):
    if path in soups:
        return soups[path]
    
    res = requests.get(f"{URL_ROOT}{path}")
    soup = BeautifulSoup(res.text)
    soups[path] = soup
    return soup

# e.g. get_category_pages("/wiki/Category:South_Levantine_Arabic_terms_with_usage_examples")
def get_category_pages(start_path):
    paths = [start_path]    
    next_path = start_path
    soup = get_soup(next_path)
    next_link = soup.find('a', string='next page')
    while next_link:
        next_path = next_link['href']
        paths.append(next_path)
        soup = get_soup(next_path)
        next_link = soup.find('a', string='next page')
    return paths

# e.g. get_lemma_pages("/wiki/Category:South_Levantine_Arabic_terms_with_usage_examples")
def get_lemma_pages(path):
    soup = get_soup(path)
    return [{'href': a['href'], 'title': a['title']} for a in soup.css.select('div#mw-pages li a')]

def get_definitions(headword, languages=['ajp', 'apc']):
    lemma = headword.strong.get_text()
    language = headword.strong.attrs['lang']
    headline = headword.find_previous(class_='mw-headline').string
    print(lemma)

    def_ol = headword.find_next_sibling('ol')
    
    results = []
    for li in def_ol.select('li'):
        result = {'lemma': lemma, 'language': language, 'headline': headline}
        definition = ''
        for s in li.strings:
            if 'dl' in list(p.name for p in s.parents):
                break
            definition += s
        result['definition'] = definition.strip()
        if li.dl:
            detail = '\n'.join(e.get_text() for e in  li.dl.find_all(class_=re.compile('^e')))
            result['detail'] = detail.strip()
        results.append(result)
    return results



In [201]:
pd.set_option('display.max_colwidth', 250)

In [168]:
# Category:Armenian_terms_with_usage_examples
# Category:Old_Armenian_terms_with_usage_examples

lps = ([lp for cp in get_category_pages('/wiki/Category:Armenian_terms_with_usage_examples')
    for lp in get_lemma_pages(cp)])
lemmas = pd.DataFrame(lps)
lemmas

,href,title
0,/wiki/%D5%A1,ա
1,/wiki/%D5%A1%CC%88,ա̈
2,/wiki/%D5%A1%CC%8A,ա̊
3,/wiki/%D5%A1%D5%A2%D5%A5%D5%A9%D5%A1%D5%BD%D5%...,աբեթասունկ
4,/wiki/%D5%A1%D5%A3%D5%BC%D5%A1%D5%BE,ագռավ
...,...,...
1789,/wiki/%D5%96%D6%80%D5%B8%D6%82%D5%B6%D5%A6%D5%A5,Ֆրունզե
1790,/wiki/%D5%9B,՛
1791,/wiki/%D5%9E,՞
1792,/wiki/%D6%89,։


In [179]:
def getLanguageSections(soup):
    headings = soup.select('div#bodyContent div.mw-heading2')
    
    heading_map = {}

    for heading in headings:
        language = heading.find('h2').text
        content = []
        for sib in heading.next_siblings:
            if sib == '\n':
                continue
            if type(sib) == bs4.element.Comment:
                continue
            if sib in headings:
                break
            content.append(sib)
        heading_map[language] = content
    return heading_map

def getSubSections(section):
    result = {}
    current_header = None
    current_subsection = []
    for tag in section:
        if tag.name == 'div' and 'mw-heading' in tag.attrs.get('class', []):
            if current_header:
                result[current_header] = current_subsection
            current_header = tag.text.replace('[edit]', '')
            current_subsection = []
        else:
            current_subsection.append(tag)
    if current_header:
        result[current_header] = current_subsection
    return result

In [203]:
len(lemmas)

1794

In [226]:
[ r[['title','href']].to_list() for _, r in lemmas.sample(1).iterrows()]

[['մանր', '/wiki/%D5%B4%D5%A1%D5%B6%D6%80']]

In [235]:
entries = {}

for title, href in tqdm([row[['title','href']].to_list() for _, row in lemmas.sample(100).iterrows()]):
    sections = getLanguageSections(get_soup(href))
    if 'Armenian' not in sections:
        continue
    entries[title] = getSubSections(sections['Armenian'])

100%|██████████| 100/100 [01:24<00:00,  1.18it/s]


In [237]:
pd.DataFrame(entries).T.count().sort_values(ascending=False).to_frame().style.bar()

,0
Pronunciation,97
Etymology,91
Declension,78
Noun,54
Alternative forms,33
Derived terms,33
Adjective,27
References,22
Related terms,17
Adverb,11


In [239]:
pd.DataFrame(entries).T #[lambda x:x['Verb'].notna()]

,Pronunciation,Verb,Inflection,Etymology,Adjective,Declension,Noun,Derived terms,Adverb,Alternative forms,...,Pronoun,Suffix,Antonyms,Synonyms,Interjection,Conjunction,Descendants,Postposition,Punctuation mark,See also
ուրախացնել,"[[[<span class=""usage-label-accent""><span class=""ib-brac"">(</span><span class=""ib-content""><a class=""extiw"" href=""https://en.wikipedia.org/wiki/Eastern_Armenian"" title=""w:Eastern Armenian"">Eastern Armenian</a></span><span class=""ib-brac"">)</span>...","[[[<strong class=""Armn headword"" lang=""hy"">ուրախացնել</strong>, , <a href=""/wiki/Wiktionary:Armenian_transliteration"" title=""Wiktionary:Armenian transliteration"">•</a>, (, <span class=""headword-tr tr Latn"" dir=""ltr"" lang=""hy-Latn"">uraxacʻnel</s...","[[\n, [causative conjugation (, <a class=""extiw"" href=""https://en.wikipedia.org/wiki/Eastern_Armenian"" title=""w:Eastern Armenian"">Eastern Armenian</a>, )], \n, [\n, <table border=""1px solid #000000"" class=""inflection-table"" style=""border-collapse...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
մսացու,"[[[], \n, [<span class=""usage-label-accent""><span class=""ib-brac"">(</span><span class=""ib-content""><a class=""extiw"" href=""https://en.wikipedia.org/wiki/Eastern_Armenian"" title=""w:Eastern Armenian"">Eastern Armenian</a></span><span class=""ib-brac"">...",NaN,NaN,"[[[<a href=""/wiki/%D5%B4%D5%AB%D5%BD#Armenian"" title=""միս"">միս</a>], , [(], [mis], [)], +‎ , [<a href=""/wiki/-%D5%A1%D6%81%D5%B8%D6%82#Armenian"" title=""-ացու"">-ացու</a>], , [(], [-acʻu], [)], .\n]]","[[[<strong class=""Armn headword"" lang=""hy"">մսացու</strong>, , <a href=""/wiki/Wiktionary:Armenian_transliteration"" title=""Wiktionary:Armenian transliteration"">•</a>, (, <span class=""headword-tr tr Latn"" dir=""ltr"" lang=""hy-Latn"">msacʻu</span>, )]...","[[\n, [<small>nominalized, <i>i</i>-type</small>, (, <a class=""extiw"" href=""https://en.wikipedia.org/wiki/Eastern_Armenian"" title=""w:Eastern Armenian"">Eastern Armenian</a>, )], \n, [\n, <table class=""inflection-table"" style=""background:#F9F9F9;t...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
գարեջուր,"[[[<span class=""usage-label-accent""><span class=""ib-brac"">(</span><span class=""ib-content""><a class=""extiw"" href=""https://en.wikipedia.org/wiki/Eastern_Armenian"" title=""w:Eastern Armenian"">Eastern Armenian</a></span><span class=""ib-brac"">)</span>...",NaN,NaN,"[[[Learned borrowing], from , [<a class=""extiw"" href=""https://en.wikipedia.org/wiki/Classical_Armenian"" title=""w:Classical Armenian"">Old Armenian</a>], , [<a class=""mw-selflink-fragment"" href=""#Old_Armenian"">գարեջուր</a>], , [(], [gareǰur], [)...",NaN,"[[\n, [<small> <i>i</i>-type, inanimate</small>, (, <a class=""extiw"" href=""https://en.wikipedia.org/wiki/Eastern_Armenian"" title=""w:Eastern Armenian"">Eastern Armenian</a>, )], \n, [\n, <table class=""inflection-table"" style=""background:#F9F9F9;te...","[[[<strong class=""Armn headword"" lang=""hy"">գարեջուր</strong>, , <a href=""/wiki/Wiktionary:Armenian_transliteration"" title=""Wiktionary:Armenian transliteration"">•</a>, (, <span class=""headword-tr tr Latn"" dir=""ltr"" lang=""hy-Latn"">gareǰur</span>,...","[[[<ul><li><span class=""Armn"" lang=""hy""><a href=""/wiki/%D5%A3%D5%A1%D6%80%D5%A5%D5%BB%D6%80%D5%A1%D5%BF%D5%B8%D6%82%D5%B6#Armenian"" title=""գարեջրատուն"">գարեջրատուն</a></span> <span class=""mention-gloss-paren annotation-paren"">(</span><span class=...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
բազմիցս,"[[[], \n, [<span class=""usage-label-accent""><span class=""ib-brac"">(</span><span class=""ib-content""><a class=""extiw"" href=""https://en.wikipedia.org/wiki/Eastern_Armenian"" title=""w:Eastern Armenian"">Eastern Armenian</a></span><span class=""ib-brac"">...",NaN,NaN,"[[From , [<a class=""extiw"" href=""https://en.wikipedia.org/wiki/Classical_Armenian"" title=""w:Classical Armenian"">Old Armenian</a>], , [<a class=""mw-selflink-fr